# Phonon lifetimes

Calculate third order properties using a ForceConstantPotential and feeding
the resulting force constants to phono3py.

In [ ]:
import json
import h5py
import subprocess
import numpy             as np
import matplotlib.pyplot as plt

from ase                     import Atoms
from ase.io                  import write
from ase.build               import bulk
from hiphive                 import ForceConstantPotential
from phonopy                 import Phonopy
from phonopy.structure.atoms import PhonopyAtoms

In [ ]:
# Read the file in JSON format to a dictionary
with open(f'setup_variables.json', 'r') as json_file:
    setup_variables = json.load(json_file)

In [ ]:
# Setup
dim        = '5 5 5'
mesh       = '14 14 14'
fontsize   = 14
markersize = 4

temperatures = [1200]
temperatures_list = ' '.join(str(T) for T in temperatures)

In [ ]:
SOLVE HOW TO GET PHONOPY

In [ ]:
# Get phono3py supercell
prim = bulk('Ni')
atoms_phonopy = PhonopyAtoms(symbols=prim.get_chemical_symbols(),
                             scaled_positions=prim.get_scaled_positions(),
                             cell=prim.cell)
phonopy = Phonopy(atoms_phonopy, supercell_matrix=dim*np.eye(3),
                  primitive_matrix=None)

### Get force constants

In [ ]:
# Load force constant potential
fcp = ForceConstantPotential.read(setup_variables['path_to_fcp'])

# Load supercell
super_cell = read_vasp(setup_variables['path_to_super_cell'])

# Get force constants for supercell
fcs = fcp.get_force_constants(super_cell)
print(fcs)

# Write force constants and POSCAR
fcs.write_to_phonopy('fc2.hdf5')
fcs.write_to_phono3py('fc3.hdf5')

### Call phono3py

In [ ]:
# Run phono3py
phono3py_cmd = f'phono3py --dim="{dim}" --fc2 --fc3 --br --mesh="{mesh}" --ts="{temperatures_list}"'
subprocess.call(phono3py_cmd, shell=True)

# Collect phono3py data
with h5py.File(f'kappa-m{mesh}.hdf5', 'r') as hf:
    temperatures = hf['temperature'][:]
    frequency = hf['frequency'][:]
    gamma = hf['gamma'][:]

### Plot results

In [ ]:
plt.figure()
plt.plot(frequency.flatten(), gamma[0].flatten(), 'o', markersize=markersize)

plt.xlabel('Frequency (THz)', fontsize=fontsize)
plt.ylabel('Gamma (THz)', fontsize=fontsize)
plt.xlim(left=0)
plt.ylim(bottom=0)

plt.title(f'T = {int(temperatures[0]):d}K')
plt.gca().tick_params(labelsize=fontsize)
plt.savefig('frequency_gamma.eps', dpi=50, bbox_inches='tight')